### Duplicate distribtion between fake and reliable

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pathlib as pl
import os

In [3]:
#from utils.mappings import labels
labels = {
    "fake": "fake",
    "bias": "fake",
    "junksci": "fake",
    "hate": "fake",
    "reliable": "reliable"
}

### New dataframe with id, orig_type, type, and duplicates=True

In [15]:
from_file = pl.Path(os.path.abspath('')).parent.parent.resolve() / "data_files/corpus/news_cleaned_2018_02_13.csv"
dups_path = pl.Path(os.path.abspath('')).parent.parent.resolve() / "data_files/corpus/duplicates.csv"
to_path = pl.Path(os.path.abspath('')).parent.parent.resolve() / "data_files/corpus/"
file_name = "duplicate_distribution.csv"

def get_dups_distribution(from_file: pl.Path, dups_path: pl.Path, to_path: pl.Path, file_name: pl.Path) -> None:
    """Get id, type and orig_type for all duplicates in source dataset.

    Writes a file to the directory
    """
    # load id and type from source file
    df = pd.read_csv(from_file, usecols=['id','type'])
    print(f"Nulls found: \n{df.isnull().sum()}")
    df = df[df['type'].notnull()]               # filter
    df = df.rename(columns={'type':'orig_type'})
    df['type'] = df['orig_type'].map(labels)

    label_counts = df['orig_type'].value_counts()

    # load id from duplicates file
    df_dups = pd.read_csv(dups_path)
    df_dups['duplicate'] = True                 # add duplicate column
    # merge source file with duplicate check
    df = df.merge(df_dups,"left",on="id")       # match duplicate IDs in source file
    df = df[df['duplicate'].notnull()]          # filter results to show only duplicate IDs and orig_type
    df.to_csv(to_path / file_name, index=False)
    print(f"\n Distribution of duplicate IDs over type was written to {to_path}/{file_name}")
    return label_counts

In [16]:
orig_label_count = get_dups_distribution(from_file, dups_path, to_path, file_name)

/var/folders/np/hj39jtf940s2chfk3bnbmsqm0000gn/T/ipykernel_14392/1657095944.py:12: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(from_file, usecols=['id','type'])


Nulls found: 
id      1654589
type    2057872
dtype: int64

 Distribution of duplicate IDs over type was written to /Users/victor/Documents/Uddannelse/Repos/fake-news/data_files/corpus/duplicate_distribution.csv


In [18]:
df = pd.read_csv(to_path / file_name)
df['orig_type'].value_counts()

bias        322657
fake        128710
reliable    103089
junksci      31784
hate          9741
Name: orig_type, dtype: int64

In [49]:
orig_labs = ["bias","fake","reliable","junksci","hate"]

pre_label_count = []
for lab in orig_labs:
    pre_label_count.append(orig_label_count[lab])

dups_label_count = []
for i in range(len(orig_labs)):
    dups_label_count.append(df['orig_type'].value_counts()[i])

data_export = []
data_export.append(orig_labs)
data_export.append(pre_label_count)
data_export.append(dups_label_count)
data_export

[['bias', 'fake', 'reliable', 'junksci', 'hate'],
 [1138998, 894746, 1913222, 117467, 76496],
 [322657, 128710, 103089, 31784, 9741]]